In [1]:
import os, sys, pprint

path_mod = os.getcwd().replace("\\src\\notebooks", "")
sys.path.insert(0,path_mod)

from src.connections import connect

context = connect.ps_connection("real_estate", "ERROR", 3)

In [ ]:
'''    
Create a Spark program to read the house data from in/RealEstate.csv,
group by location, aggregate the average price per SQ Ft and sort by average price per SQ Ft.

The houses dataset contains a collection of recent real estate listings in 
San Luis Obispo county and around it. 

The dataset contains the following fields:
1. MLS: Multiple listing service number for the house (unique ID).
2. Location: city/town where the house is located. Most locations are in 
    San Luis Obispo county and northern Santa Barbara county (Santa Maria­Orcutt, Lompoc, 
    Guadelupe, Los Alamos), but there some out of area locations as well.
3. Price: the most recent listing price of the house (in dollars).
4. Bedrooms: number of bedrooms.
5. Bathrooms: number of bathrooms.
6. Size: size of the house in square feet.
7. Price/SQ.ft: price of the house per square foot.
8. Status: type of sale. Thee types are represented in the dataset: Short Sale, 
    Foreclosure and Regular.

Each field is comma separated.

Sample output:

+----------------+-----------------+
|        Location| avg(Price SQ Ft)|
+----------------+-----------------+
|          Oceano|             95.0|
|         Bradley|            206.0|
| San Luis Obispo|            359.0|
|      Santa Ynez|            491.4|
|         Cayucos|            887.0|
|................|.................|
|................|.................|
|................|.................|
'''

In [13]:
file_path = f"{os.environ['PROJECT_HOME']}/python-spark-tutorial/in/RealEstate.csv"
SQ_FT = 'Price SQ Ft'

real_estate = context.sql.read.csv(file_path, header=True, inferSchema=True)

In [21]:
real_estate.printSchema()

root
 |-- MLS: integer (nullable = true)
 |-- Location: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Bedrooms: integer (nullable = true)
 |-- Bathrooms: integer (nullable = true)
 |-- Size: integer (nullable = true)
 |-- Price SQ Ft: double (nullable = true)
 |-- Status: string (nullable = true)



In [22]:
output = (
    real_estate
    .groupBy(real_estate.Location)
    .avg(SQ_FT)
    .orderBy("avg(Price SQ FT)")
)

output.show()

+-------------------+------------------+
|           Location|  avg(Price SQ Ft)|
+-------------------+------------------+
|         New Cuyama|             34.05|
|        Bakersfield|             69.69|
|          King City| 71.51333333333334|
|         Greenfield|             91.58|
|    Santa Margarita|             95.38|
|            Soledad|102.69333333333333|
|        Out Of Area|116.23333333333333|
|          Guadalupe|           120.175|
|           Coalinga|124.34285714285714|
| Santa Maria-Orcutt|147.58871698113194|
|             Lompoc|             149.9|
|             Lompoc|159.87115384615387|
|         San Miguel|163.16071428571425|
|            Bradley|            166.81|
|            Creston|            181.76|
| Santa Maria-Orcutt|183.03692307692307|
|             Nipomo|187.92333333333332|
|        Paso Robles|191.17752941176474|
|         Los Alamos|191.99333333333334|
|            Solvang|           193.305|
+-------------------+------------------+
only showing top